# Notebook 3 : SQL

In [1]:
# Décommenter la ligne suivante pour installer ibis
%pip install ibis-framework[sqlite]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sqlite3

import pandas as pd
import ibis

from ibis import _

ibis.options.interactive = True

query_tables = "SELECT name FROM sqlite_master WHERE type='table'"

## STAR

Nous considérons les données des stations de vélos en libre service [STAR](https://www.star.fr/) de Rennes Métropole. Une copie de la base SQLite est disponible dans le fichier `star.db`. Nous utilisons d'abord Pandas pour répondre aux questions, puis Ibis.

1. Se connecter à la base de données et afficher la liste des tables à l'aide de la fonction `read_sql` de Pandas et de la requête `query_tables`.

In [3]:
con = sqlite3.connect("data\\star.db")
pd.read_sql(query_tables, con)

,name
0,Topologie
1,Etat


2. Récupérer le contenu de la table `Etat` dans un dataframe et afficher la liste des variables disponibles. Même question pour la table `Topologie`.

In [5]:
df_etat = pd.read_sql("SELECT * FROM Etat", con)
print(df_etat.dtypes)
df_topologie = pd.read_sql("SELECT * FROM Topologie", con)
print(df_topologie.dtypes)

id                            int64
nom                          object
latitude                    float64
longitude                   float64
etat                         object
nb_emplacements               int64
emplacements_disponibles      int64
velos_disponibles             int64
date                        float64
data                         object
dtype: object
id                     int64
nom                   object
adresse_numero        object
adresse_voie          object
commune               object
latitude             float64
longitude            float64
id_correspondance    float64
mise_en_service      float64
nb_emplacements        int64
id_proche_1            int64
id_proche_2            int64
id_proche_3            int64
terminal_cb           object
dtype: object


3. Sélectionner l'identifiant `id`, le nom `nom` et l'identifiant de la station la plus proche `id_proche_1` depuis la table `Topologie`.

In [8]:
query_base = """
SELECT
id,
nom,
id_proche_1
FROM Topologie
"""

print(
pd.read_sql(query_base, con))
proche_df=pd.read_sql(query_base, con)

    id                nom  id_proche_1
0    1         République            2
1    2             Mairie            1
2    3      Champ Jacquet            2
3   10   Musée Beaux-Arts           12
4   12                TNB           10
..  ..                ...          ...
78  62         Clemenceau           63
79  66  Bréquigny Piscine           65
80  69    Champs Manceaux           66
81  85       La Courrouze           20
82  86          Armorique           78

[83 rows x 3 columns]


4. Faire une jointure sur la table précédente pour créer une table qui contient la liste des stations avec l'identifiant, le nom et le nom de la station la plus proche associée à l'identifiant `id_proche_1`. Les variables utilisées comme clés sont différents, penser à utiliser les arguments `left_on` et `right_on` de la méthode `merge`.

In [18]:
(
    proche_df
    .merge(df_topologie[["id", "nom"]], how="left", left_on = "id_proche_1", right_on="id")
    .filter(items=["id_x", "nom_x", "nom_y"])
    .rename(columns={
        "id_x" : "id",
        "nom_x" : "nom",
        "nom_y" : "nom_proche_1",
    })
)

,id,nom,nom_proche_1
0,1,République,Mairie
1,2,Mairie,République
2,3,Champ Jacquet,Mairie
3,10,Musée Beaux-Arts,TNB
4,12,TNB,Musée Beaux-Arts
...,...,...,...
78,62,Clemenceau,Henri Fréville
79,66,Bréquigny Piscine,NaN
80,69,Champs Manceaux,Bréquigny Piscine
81,85,La Courrouze,Pont de Nantes


5. Ajouter à la table précédente la distance entre la station et la station la plus proche.

In [27]:
station_df = (
    proche_df
    .merge(df_topologie[["id", "nom", "latitude", "longitude"]], how="left", left_on = "id_proche_1", right_on="id")
    .filter(items=["id_x", "nom_x", "nom_y", "latitude", "longitude"])
    .rename(columns={
        "id_x" : "id",
        "nom_x" : "nom",
        "nom_y" : "nom_proche_1"
    })
    .merge(df_topologie[["id", "latitude", "longitude"]], how="left", left_on = "id", right_on="id", suffixes=["", "_proche_1"])
)
station_df["distance"] = (station_df.latitude - station_df.latitude_proche_1)**2 + (station_df.longitude - station_df.longitude_proche_1)**2
print(station_df)

    id                nom       nom_proche_1   latitude  longitude  \
0    1         République             Mairie  48.111624  -1.678757   
1    2             Mairie         République  48.110026  -1.678037   
2    3      Champ Jacquet             Mairie  48.111624  -1.678757   
3   10   Musée Beaux-Arts                TNB  48.107748  -1.673711   
4   12                TNB   Musée Beaux-Arts  48.109601  -1.674080   
..  ..                ...                ...        ...        ...   
78  62         Clemenceau     Henri Fréville  48.087337  -1.674849   
79  66  Bréquigny Piscine                NaN        NaN        NaN   
80  69    Champs Manceaux  Bréquigny Piscine  48.089621  -1.690242   
81  85       La Courrouze     Pont de Nantes  48.102015  -1.684015   
82  86          Armorique         Gros-Chêne  48.126795  -1.665101   

    latitude_proche_1  longitude_proche_1  distance  
0           48.110026           -1.678037  0.000003  
1           48.111624           -1.678757  0.000003

6. Créer une table avec le nom des trois stations les plus proches du point GPS *(48.1179151,-1.7028661)* classées par ordre de distance et le nombre de vélos disponibles dans ces stations.

In [33]:
df_etat["distance_point"] = (df_etat.latitude - 48.1179151)**2 + (df_etat.longitude + 1.7028661)**2
print(df_etat)

    id                nom   latitude  longitude               etat  \
0    1         République  48.110026  -1.678037  En fonctionnement   
1    2             Mairie  48.111624  -1.678757  En fonctionnement   
2    3      Champ Jacquet  48.112764  -1.680062  En fonctionnement   
3   10   Musée Beaux-Arts  48.109601  -1.674080  En fonctionnement   
4   12                TNB  48.107748  -1.673711  En fonctionnement   
..  ..                ...        ...        ...                ...   
78  62         Clemenceau  48.093292  -1.674116  En fonctionnement   
79  66  Bréquigny Piscine  48.089621  -1.690242  En fonctionnement   
80  69    Champs Manceaux  48.091114  -1.682284  En fonctionnement   
81  85       La Courrouze  48.098909  -1.694523  En fonctionnement   
82  86          Armorique  48.129754  -1.675788  En fonctionnement   

    nb_emplacements  emplacements_disponibles  velos_disponibles  \
0                30                        25                  5   
1                24    

In [41]:
df_etat.sort_values(by=["distance_point", "velos_disponibles"]).head(3)


,id,nom,latitude,longitude,etat,nb_emplacements,emplacements_disponibles,velos_disponibles,date,data,distance_point
50,56,Berger,48.116314,-1.705097,En fonctionnement,14,4,10,1.524646e+09,2018-04-25 08:47:04,0.000008
17,52,Villejean-Université,48.121076,-1.704122,En fonctionnement,24,13,11,1.524646e+09,2018-04-25 08:47:04,0.000012
74,38,Marbeuf,48.111749,-1.702077,En fonctionnement,19,10,9,1.524646e+09,2018-04-25 08:47:04,0.000039


7. Reprendre les questions précédentes en utilisant le module `ibis`. Pour les jointures, utiliser la méthode `left_join`.

In [36]:
con = ibis.sqlite.connect("data\\star.db")
con.list_tables()

['Etat', 'Topologie']

In [37]:
etat = con.table("Etat")
etat.columns

('id',
 'nom',
 'latitude',
 'longitude',
 'etat',
 'nb_emplacements',
 'emplacements_disponibles',
 'velos_disponibles',
 'date',
 'data')

In [38]:
topologie = con.table("Topologie")
topologie.columns

('id',
 'nom',
 'adresse_numero',
 'adresse_voie',
 'commune',
 'latitude',
 'longitude',
 'id_correspondance',
 'mise_en_service',
 'nb_emplacements',
 'id_proche_1',
 'id_proche_2',
 'id_proche_3',
 'terminal_cb')

In [51]:
topologie_ibis = topologie.select("id", "nom", "id_proche_1", "latitude", "longitude")

In [58]:
from ibis import _

(
    topologie_ibis
    .left_join(topologie[["id", "nom", "latitude", "longitude"]], topologie_ibis.id_proche_1 == topologie[["id", "nom", "latitude", "longitude"]].id)
    .select(["id", "nom", "latitude", "longitude", "nom_right", "latitude_right", "longitude_right" ])
)

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ id    ┃ nom                ┃ latitude  ┃ longitude ┃ nom_right          ┃ latitude_right ┃ longitude_right ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ int64 │ string             │ float64   │ float64   │ string             │ float64        │ float64         │
├───────┼────────────────────┼───────────┼───────────┼────────────────────┼────────────────┼─────────────────┤
│     1 │ République         │ 48.110026 │ -1.678037 │ Mairie             │      48.111624 │       -1.678757 │
│     2 │ Mairie             │ 48.111624 │ -1.678757 │ République         │      48.110026 │       -1.678037 │
│     3 │ Champ Jacquet      │ 48.112764 │ -1.680062 │ Mairie             │      48.111624 │       -1.678757 │
│    10 │ Musée Beaux-Arts   │ 48.109601 │ -1.674080 │ TNB                │      48.107748 │       -1.673711 │
│    12 │ TNB                │ 48.107748 │ -1.673711 │ Musée Beaux-Arts   │      48.109601 │       -1.674080 │
│    14 │ Laënnec            │ 48.106847 │ -1.665817 │ Pont de Châteaudun │      48.110264 │       -1.664443 │
│    17 │ Charles de Gaulle  │ 48.105111 │ -1.677119 │ Champs Libres      │      48.105537 │       -1.674328 │
│    20 │ Pont de Nantes     │ 48.102015 │ -1.684015 │ Cité Judiciaire    │      48.104771 │       -1.684110 │
│    22 │ Oberthur           │ 48.113550 │ -1.661858 │ Metz - Sévigné     │      48.115993 │       -1.658258 │
│    25 │ Office de Tourisme │ 48.110294 │ -1.683106 │ Place de Bretagne  │      48.109621 │       -1.684019 │
│     … │ …                  │         … │         … │ …                  │              … │               … │
└───────┴────────────────────┴───────────┴───────────┴────────────────────┴────────────────┴─────────────────┘

8. (*Bonus*) Écrire des requêtes SQL pour obtenir les résultats demandés dans les questions 3 à 6. La fonction `to_sql` pourra être utilisée pour de l'aide.

## Musique

Le dépôt GitHub [lerocha/chinook-database](https://github.com/lerocha/chinook-database) met à disposition des bases de données de bibliothèques musicales. Une copie de la base SQLite est disponible dans le fichier `chinook.db`.

1. Utiliser le module `ibis` pour vous connecter à la base de données et explorer les tables formant le jeu de données pour le découvrir. En particulier, remarquer comment les tables `Playlist`, `PlaylistTrack` et `Track` sont liées entre elles.

In [1]:
import sqlite3

import pandas as pd
import ibis

from ibis import _

ibis.options.interactive = True

In [2]:
con = ibis.sqlite.connect("data\\chinook.db")
con.list_tables()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [3]:
playlist = con.table("Playlist")
playlist.columns

('PlaylistId', 'Name')

In [4]:
playlist_track = con.table("PlaylistTrack")
playlist_track.columns

('PlaylistId', 'TrackId')

In [5]:
track = con.table("Track")
track.columns

('TrackId',
 'Name',
 'AlbumId',
 'MediaTypeId',
 'GenreId',
 'Composer',
 'Milliseconds',
 'Bytes',
 'UnitPrice')

2. Quelles sont les playlists qui contiennent le plus de pistes ?

In [6]:
(
    playlist_track
    .group_by("PlaylistId")
    .aggregate(n_tracks = playlist_track.TrackId.count())
    .left_join(playlist, "PlaylistId")
    .select(["Name", "n_tracks"])
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Name                       ┃ n_tracks ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ string(120)                │ int64    │
├────────────────────────────┼──────────┤
│ Music                      │     3290 │
│ TV Shows                   │      213 │
│ 90’s Music                 │     1477 │
│ Music                      │     3290 │
│ Music Videos               │        1 │
│ TV Shows                   │      213 │
│ Brazilian Music            │       39 │
│ Classical                  │       75 │
│ Classical 101 - Deep Cuts  │       25 │
│ Classical 101 - Next Steps │       25 │
│ …                          │        … │
└────────────────────────────┴──────────┘

3. Construire une table contenant les informations suivantes sur la playlist `Classical` : le titre de chaque piste ainsi que le titre de l'album dont cette piste est tirée.

In [8]:
album = con.table("Album")

(
    playlist_track
    .left_join(playlist, playlist_track.PlaylistId == playlist.PlaylistId)
    .rename(PlaylistName="Name")
    .filter(_.PlaylistName == "Classical")
    .left_join(track, _.TrackId == track.TrackId)
    .left_join(album, _.AlbumId == album.AlbumId)
    .select("Name", "Title")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                                                                 ┃ Title                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string(200)                                                          │ string(160)                                               │
├──────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│ Intoitus: Adorate Deum                                               │ Adorate Deum: Gregorian Chant from the Proper of the Mass │
│ Miserere mei, Deus                                                   │ Allegri: Miserere                                         │
│ Canon and Gigue in D Major: I. Canon                                 │ Pachelbel: Canon & Gigue                                  │
│ Concerto No. 1 in E Major, RV 269 "Spring": I. Allegro               │ Vivaldi: The Four Seasons                                 │
│ Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace               │ Bach: Violin Concertos                                    │
│ Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria       │ Bach: Goldberg Variations                                 │
│ Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude          │ Bach: The Cello Suites                                    │
│ The Messiah: Behold, I Tell You a Mystery... The Trumpet Shall Sound │ Handel: The Messiah (Highlights)                          │
│ Solomon HWV 67: The Arrival of the Queen of Sheba                    │ The World of Classical Favourites                         │
│ "Eine Kleine Nachtmusik" Serenade In G, K. 525: I. Allegro           │ Sir Neville Marriner: A Celebration                       │
│ …                                                                    │ …                                                         │
└──────────────────────────────────────────────────────────────────────┴───────────────────────────────────────────────────────────┘

4. (*Bonus*) Écrire une requête SQL donnant le résultat de la question précédente. La fonction `to_sql` pourra être utilisée pour de l'aide.